<a href="https://colab.research.google.com/github/kwakyosong/KOSA_KDT/blob/main/KDT_day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>